<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/KoBERTopic_%EB%B6%88%EC%9A%A9%EC%96%B4%EC%A0%9C%EA%B1%B0_newDiscriminataion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install bertopic[visualization]

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install bertopic

In [ ]:
%pip install bertopic[visualization]

# 필요한 것들을 임포트

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab, Okt
from bertopic import BERTopic

Okt = Okt()

# 데이터 불러오기

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
documents = [line.strip() for line in open("newDiscrimination_text.txt", encoding="utf-8").readlines()]
documents[:5]

['못사는 놈들은 정말 불쌍해.',
 '쟤처럼 느글거리는 인상이 꼭 사고치더라',
 '어린 애들 따 먹어봤다는 거잖아 좀 부럽네',
 '너 계속 이러면 나중에 청소차 몰고 다닌다고 엄마가 말했어 안했어',
 '와 미국인 정서 걍 이해불가던데 어떻게 사귀는거지?']

# Mecab과 SBERT를 이용한 Bertopic (형태소단위)

In [ ]:
# 불용어 사전 만들기
korean_stopwords_path = "./korean_stopwords.txt"

with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        words = [word for word in word_tokens if len(word) > 1]

        result = []  # 불용어 제거
        for w in words: 
         if w not in stopwords: 
            result.append(w)

        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

## 토픽개수 8개로 지정

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=8,
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(documents)

In [ ]:
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('한국', 0.20680347475598773), ('중국', 0.1907380840126766), ('조선족', 0.1760029572391385), ('서울', 0.10436686293459244), ('중국인', 0.10198267660872008), ('한국인', 0.06941121654818305), ('일본', 0.06628873979566804), ('사람', 0.06617663908591379), ('북한', 0.05954657180404848), ('나라', 0.04879527646499842)]
1 번째 토픽 : [('여자', 0.09925279269353482), ('남자', 0.06840351296848349), ('사람', 0.04341853433799328), ('ㅋㅋ', 0.040166599219244106), ('는데', 0.035632928438106844), ('잖아', 0.033830822606877815), ('으면', 0.032715998385594386), ('한테', 0.027318726312579422), ('ㅋㅋㅋ', 0.026245819605764102), ('니까', 0.02585522311726267)]
2 번째 토픽 : False
3 번째 토픽 : False
4 번째 토픽 : False
5 번째 토픽 : False
6 번째 토픽 : False
7 번째 토픽 : False


In [ ]:
model.visualize_barchart()

## 토픽개수 지정 안함

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 top_n_words=10,
                 calculate_probabilities=True)

topics, probs = model.fit_transform(documents)

for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('ㅋㅋ', 0.020702528916565044), ('짱깨', 0.020363333729879456), ('남자', 0.020042654960572004), ('..', 0.019750072378182642), ('전라도', 0.018327000356891512), ('그렇', 0.018121676952837586), ('사람', 0.016774479969701404), ('그게', 0.015455753835726928), ('장애', 0.014055221771142426), ('한테', 0.014027872571457574)]
1 번째 토픽 : [('ㅋㅋ', 0.025659472459967945), ('는데', 0.024001377763337207), ('이나', 0.02134218663323906), ('아이돌', 0.017937323493655587), ('라고', 0.01606536043731372), ('남자', 0.015882334755179663), ('면서', 0.015690071876793502), ('여자', 0.015597898539510336), ('어서', 0.013042876693762124), ('회사', 0.012724245518296139)]
2 번째 토픽 : [('ㅋㅋ', 0.027175322953345234), ('ㅋㅋㅋ', 0.02284771512424907), ('여자', 0.019251941755807304), ('니까', 0.01816521372797007), ('는데', 0.0177200539165265), ('남자', 0.016820594300935438), ('어서', 0.015718688991162192), ('으면', 0.014221070191886781), ('더라', 0.014129313082914055), ('ooo', 0.013969707484707924)]
3 번째 토픽 : [('한국', 0.13805000209871976), ('서울', 0.10268140300792135), 

In [ ]:
model.visualize_barchart(top_n_topics = 20)

- 토픽개수를 지정하거나 그렇지 않았을때 반복해서 돌려보니 2-4개의 토픽으로 모델링 되는 경우가 있다.

## nr_topics="auto" 자동 토픽 축소 사용 (모델이 자동으로 토픽의 수를 줄이도록 설정)

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics="auto",
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('ㅋㅋ', 0.02152907873329928), ('짱깨', 0.02083968286753741), ('..', 0.02006204950282891), ('그렇', 0.01953455426141412), ('전라도', 0.01864603203937558), ('남자', 0.01847838934566042), ('그게', 0.016704978329323307), ('사람', 0.016346739038791934), ('그건', 0.01505319320605078), ('한테', 0.014249490331484928)]
1 번째 토픽 : [('ㅋㅋ', 0.026712990897387), ('는데', 0.023590448648050318), ('라고', 0.017697379979262785), ('아이돌', 0.017071952671036936), ('남자', 0.015691538426237654), ('여자', 0.015608721448461055), ('해서', 0.01488394760138634), ('면서', 0.014398368842769436), ('어서', 0.013699465294359987), ('이나', 0.013402713546871625)]
2 번째 토픽 : [('이래서', 0.02907443554111536), ('들어가', 0.025823022223380787), ('이나', 0.025033672368458798), ('이랑', 0.02416078464973104), ('는데', 0.024034029733740154), ('전라도', 0.023948917303464795), ('남자', 0.021886407076400667), ('후군', 0.02096271272836936), ('운증', 0.02096271272836936), ('한테', 0.019445885666043536)]
3 번째 토픽 : [('ㅋㅋ', 0.029934481025400157), ('는데', 0.026236609278072136), ('ㅋㅋㅋ'

In [ ]:
model.visualize_barchart(top_n_topics = 20)

- 명사가 아닌 단어들은 의미가 없어보인다.

# 명사단위 분석

## 토픽개수 8개로 설정

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.nouns(sent)

        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
           if w not in stopwords: 
             result.append(w)
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=8,
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('짱깨', 0.06187652837489286), ('남자', 0.059442728884659575), ('사람', 0.048736270792982386), ('전라도', 0.03992034088702765), ('게이', 0.035352647255552344), ('여자', 0.03453971736521918), ('장애', 0.03105169385501886), ('수준', 0.02921061163914053), ('그게', 0.028682059453091466), ('그건', 0.02479241850180271)]
1 번째 토픽 : [('여자', 0.07211574203108119), ('남자', 0.056662550352431076), ('길거리', 0.03418451984578694), ('큰일', 0.03180451355491531), ('차이', 0.0295661867903934), ('사람', 0.02908746746174872), ('흑인', 0.02807741531567622), ('방송인', 0.02732596463049863), ('장애', 0.025165468297185564), ('수준', 0.02498857365432358)]
2 번째 토픽 : [('한국', 0.2825517023822703), ('조선족', 0.20618637741408913), ('서울', 0.1849921636315542), ('한국인', 0.11207788052938661), ('북한', 0.08156761019471814), ('인천', 0.0667444538388616), ('한국어', 0.055585291339291756), ('사람', 0.055416640595228175), ('나라', 0.05437130040373913), ('대한민국', 0.041688968504468814)]
3 번째 토픽 : [('전라도', 0.0846643896312378), ('환자', 0.07451998560481665), ('볼펜', 0.070369

In [ ]:
model.visualize_barchart()

## 토픽 개수 미설정

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('짱깨', 0.031119614411778962), ('전라도', 0.028007652970601062), ('그게', 0.024385259302431076), ('남자', 0.02319227285436864), ('사람', 0.02057302672207253), ('그건', 0.01994142410491602), ('지역', 0.01744874609180152), ('엄마', 0.017402798775693604), ('홍어', 0.016700786628193236), ('수준', 0.01669529528698539)]
1 번째 토픽 : [('아이돌', 0.025693595439791363), ('여자', 0.020184054929676495), ('회사', 0.019955064874217457), ('남자', 0.018356447797956078), ('취직', 0.016819060149593147), ('공부', 0.01663374294620861), ('친구', 0.01644424742143033), ('출신', 0.015300328891238654), ('여친', 0.015191618908342292), ('사람', 0.014990371138608734)]
2 번째 토픽 : [('한국', 0.1167995307623605), ('중국', 0.10685914516556387), ('조선족', 0.09940385596796639), ('중국인', 0.06990055371805501), ('서울', 0.0674729482307094), ('한국인', 0.05199861883141971), ('북한', 0.045729983647731766), ('일본', 0.044738465129838305), ('세계', 0.028273727717662605), ('인천', 0.02748415322336892)]
3 번째 토픽 : [('길거리', 0.026538701108733357), ('남자', 0.02418235970284142), ('큰일', 

In [ ]:
model.visualize_barchart(top_n_topics=10)

## nr_topics="auto" 자동 토픽 축소 사용

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics="auto",
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(len(set(topics))):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('여자', 0.10317035971552875), ('남자', 0.07730793966681354), ('사람', 0.05224129078377382), ('장애', 0.032812953291697027), ('흑인', 0.02639500133151897), ('얼굴', 0.025424341993441457), ('생각', 0.024806150679675433), ('결혼', 0.024644007481914938), ('나라', 0.022652596154336326), ('전라도', 0.021101140100325225)]
1 번째 토픽 : [('한국', 0.3334898100294537), ('조선족', 0.2308775607896218), ('서울', 0.19017899971272403), ('북한', 0.12389684110989418), ('한국인', 0.11966983048676953), ('사람', 0.07299690446628837), ('인천', 0.06999837327962373), ('한국어', 0.058074694672178864), ('나라', 0.0541555127831615), ('대한민국', 0.04355602100413415)]
2 번째 토픽 : [('중국', 0.731329842132285), ('중국인', 0.562683860222886), ('베트남', 0.11053730847960212), ('유학', 0.1019141276138381), ('세계', 0.09975539698697167), ('이중국', 0.08491621477605747), ('적자', 0.08491621477605747), ('개국', 0.07994723751882074), ('짱꼴라', 0.07994723751882074), ('공산당', 0.07642344789682434)]
3 번째 토픽 : [('일본', 0.8893130311622907), ('일본인', 0.5315174483614461), ('나라', 0.2489219462

In [ ]:
model.visualize_barchart()

In [ ]:
model.visualize_topics()

- BERTopic을 적용하니 토픽의 개수가 2-4개로 나오는 경우도 꽤 있다.